<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250617_BigQuery%E6%B8%ACTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-cloud-bigquery

In [11]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證 # 此為示範，實際上應放在環境變數裡面(要隱藏起來)
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad251-31-0613-c339499c7e0e.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


In [12]:
# 定義變數
dataset_id = 'tibame_gad251_31_dataset_py'  # 替換為想要建立的資料集名稱
project_id = 'tibame-gad251-31-0613'  # 替換為 Google Cloud 專案 ID
table_id = 'age_table'


In [13]:
# 定義 SQL 查詢以創建模型
query = f"""
CREATE OR REPLACE MODEL `{project_id}.{dataset_id}.imported_tf_model`
OPTIONS (MODEL_TYPE='TENSORFLOW',
         MODEL_PATH='gs://cloud-training-demos/txtclass/export/exporter/1549825580/*')
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("TensorFlow model import BigQuery ML")


TensorFlow model import BigQuery ML


In [14]:
# 定義 SQL 查詢以使用模型進行預測，只取五筆非 NULL 的資料
query = f"""
SELECT *
FROM ML.PREDICT(MODEL `{project_id}.{dataset_id}.imported_tf_model`,
  (SELECT title AS input
   FROM `bigquery-public-data.hacker_news.full`
   WHERE title IS NOT NULL
   LIMIT 5)  -- 限制結果為五筆資料
)
"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)


Row(([0.868116021156311, 0.0698704645037651, 0.06201356276869774], 'Azure SQL Database Automatic Tuning'), {'dense_1': 0, 'input': 1})
Row(([1.3162051573090139e-06, 0.9999954700469971, 3.191369160049362e-06], 'How Inoreader Migrated from Bare-Metal Servers to OpenNebula and StorPool'), {'dense_1': 0, 'input': 1})
Row(([2.4120627717394427e-09, 0.2178039401769638, 0.782196044921875], 'Trump organization charges Iranians for worldwide digital burglary'), {'dense_1': 0, 'input': 1})
Row(([0.00048323272494599223, 0.9993659853935242, 0.00015083025209605694], 'World’s Wealthiest Lose $71B in Market Selloff'), {'dense_1': 0, 'input': 1})
Row(([0.00029113070922903717, 0.9997060894966125, 2.715593836910557e-06], 'Elon Musk Just Removed Tesla and SpaceX from Facebook'), {'dense_1': 0, 'input': 1})


In [15]:
# 定義資料集參數
dataset_ref = client.dataset(dataset_id)

# 刪除資料集
client.delete_dataset(dataset_ref, delete_contents=True)
print(f"dataset: {dataset_id} ,deleted")

dataset: tibame_gad251_31_dataset_py ,deleted
